In [1]:
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
import pickle
import pandas as pd
from sklearn.metrics import accuracy_score
import time

%matplotlib inline

In [2]:
with open('./data/mnist/mnist_X_train.p', 'rb') as f:
    X_train = pickle.load(f)
with open('./data/mnist/mnist_y_train.p', 'rb') as f:
    y_train = pickle.load(f)
with open('./data/mnist/mnist_X_test.p', 'rb') as f:
    X_test = pickle.load(f)
with open('./data/mnist/mnist_y_test.p', 'rb') as f:
    y_test = pickle.load(f)

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import transforms

class DatasetMNIST(Dataset):
    
    def __init__(self, X_path, y_path, transform=None):
        with open(X_path, 'rb') as f:
            self.X = pickle.load(f)
        with open(y_path, 'rb') as f:
            self.y = pickle.load(f)
        self.transform = transform
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        image = self.X[index].astype(np.uint8).reshape((28, 28, 1))
        label = self.y[index]
        
        if self.transform is not None:
            image = self.transform(image*255)
            label = torch.tensor(label, dtype = torch.long)
        return image, label

In [4]:
test_dataset = DatasetMNIST('./data/mnist/mnist_X_test.p','./data/mnist/mnist_y_test.p', transform=transforms.ToTensor())
image,label = test_dataset.__getitem__(0)

print('images shape on batch size = {}'.format(image.shape))
print('labels shape on batch size = {}'.format(label))

images shape on batch size = torch.Size([1, 28, 28])
labels shape on batch size = 1


In [9]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 5, 1, padding=2)
        self.conv2 = nn.Conv2d(64, 32, 5, 1, padding=1)
        self.fc1 = nn.Linear(32 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)
        self.dropout = nn.Dropout(p=0.2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 32 * 6 * 6)
        x = F.relu(self.fc1(self.dropout(x)))
        x = F.relu(self.fc2(self.dropout(x)))
        x = self.fc3(self.dropout(x))
        return F.log_softmax(x, dim=1)
    
def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f})\n'.format(
        test_loss, correct, len(test_loader.dataset),
        1. * correct / len(test_loader.dataset)))

In [10]:
torch.cuda.is_available()

True

In [11]:
use_cuda = torch.cuda.is_available()
#use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

batch_size = 64
test_batch_size = 1000 
epochs = 6
save_model = False

lr = 0.01
momentum = 0.5

start = time.time()

kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    DatasetMNIST('./data/mnist/mnist_X_train.p', './data/mnist/mnist_y_train.p',
                    transform = transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    DatasetMNIST('./data/mnist/mnist_X_test.p', './data/mnist/mnist_y_test.p',
                    transform = transforms.ToTensor()),
    batch_size=test_batch_size, shuffle=True, **kwargs)

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval = 10)
    test(model, device, test_loader)
    
test(model, device, train_loader)

end = time.time() - start
print("Time spent : {:4f}s".format(end))

if (save_model):
    torch.save(model.state_dict(),"mnist_cnn.pt")

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.690812
Train Epoch: 1 [640/60000 (1%)]	Loss: 0.682594
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.680900
Train Epoch: 1 [1920/60000 (3%)]	Loss: 0.690217
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.680580
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.697681
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.687509
Train Epoch: 1 [4480/60000 (7%)]	Loss: 0.684934
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.688760
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.688929
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.688983
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.691727
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.687355
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.685057
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.685450
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.682440
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.683513
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.679492
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.678855
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.671429
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.121289
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.090714
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.096626
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.102810
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.026883
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.058112
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.020829
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.110762
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.078665
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.051848
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.062421
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.097235
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.080127
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.122540
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.132176
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.071838
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.143721
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.070844
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.078633
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.120489


Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.055545
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.034258
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.020156
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.050289
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.005739
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.022643
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.105849
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.015569
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.093606
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.085082
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.030060
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.068777
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.056636
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.030734
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.052877
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.040180
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.044967
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.048711
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.045740
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.029312


Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.114760
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.014501
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.044719
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.015401
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.090600
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.018978
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.034943
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.007831
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.017044
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.037890
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.039688
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.062512
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.043878
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.021861
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.025401
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.074476
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.012226
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.132510
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.083895
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.063897
